In [5]:
# Load data into a Pandas dataframe
import pandas as pd
df=pd.read_excel('new_qa.xlsx')
df.columns

Index(['source_text', 'target_text'], dtype='object')

In [6]:
df=df.reset_index()
df

,index,source_text,target_text
0,0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul..."
1,1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117..."
2,2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489..."
3,3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125..."
4,4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549..."
...,...,...,...
95,95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052..."
96,96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016..."
97,97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."
98,98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."


In [7]:
df=df.drop('index',axis=1)

In [90]:
df = df.rename(columns={"target_text":"summary", "source_text":"text"})
df

,text,summary
0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul..."
1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117..."
2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489..."
3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125..."
4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549..."
...,...,...
95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052..."
96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016..."
97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."
98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."


In [71]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1", num_labels=2)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/dmis-lab/biobert-base-cased-v1.1/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\78e6e8ece5b58501028ce314273009ad7707ef4c5ba44251914fd6bca8a05eff.e4a2e693122d98b8b56b7dc1f0d89b644226aacef228afb5030ee3621b2829d3
Model config BertConfig {
  "_name_or_path": "dmis-lab/biobert-base-cased-v1.1",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading

In [72]:
# Load your data into a pandas DataFrame
 

# Split your data into training and validation sets
train_size = int(0.5 * len(df))
train_dataset = df[:train_size]
val_dataset = df[train_size:]


In [73]:
def encode_data(data):
    # Encode the text and labels
    text = data['text'].tolist()
    labels = data['label'].tolist()

    # Tokenize the text
    encoded = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Create input features for the model
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    labels = torch.tensor(labels)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


In [74]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    data_collator=encode_data,           # function to encode the data
)

# Start the training
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
C:\Users\AlexandruDaia\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21


KeyError: 14

In [86]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, EncoderDecoderConfig, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
config = EncoderDecoderConfig.from_encoder_decoder_pretrained("dmis-lab/biobert-base-cased-v1.1", "dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSeq2SeqLM.from_pretrained("dmis-lab/biobert-base-cased-v1.1", config=config)

# Load your data into a pandas DataFrame
#df = pd.read_csv('your_data.csv')

# Define a function to preprocess the data
def preprocess_data(data):
    # Tokenize the input
    inputs = tokenizer(data['text'], padding='max_length', truncation=True, max_length=512)

    # Tokenize the label
    labels = tokenizer(data['label'], padding='max_length', truncation=True, max_length=150)

    # Convert the labels to a list
    label_ids = labels['input_ids'].tolist()

    # Replace -100 values in the labels with the pad token
    for i, label_id in enumerate(label_ids):
        for j, token_id in enumerate(label_id):
            if token_id == -100:
                label_ids[i][j] = tokenizer.pad_token_id

    # Return the processed inputs and labels
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'],
            'decoder_input_ids': labels['input_ids'][:, :-1], 'decoder_attention_mask': labels['attention_mask'][:, :-1],
            'labels': label_ids}


# Split your data into training and validation sets
train_size = int(0.8 * len(df))
train_dataset = df[:train_size]
val_dataset = df[train_size:]

# Prepare the training and validation data
train_encodings = train_dataset.apply(preprocess_data, axis=1)
val_encodings = val_dataset.apply(preprocess_data, axis=1)

# Define the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='steps',     # evaluate every eval_steps
    eval_steps=500,                  # evaluation and save happens every 500 steps
    save_total_limit=3,              # Only last 3 models are saved. Older ones are deleted.
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_encodings,       # training dataset
    eval_dataset=val_encodings,          # evaluation dataset
    data_collator=data_collator,          # data collator
)

# Start the training
trainer.train()


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/dmis-lab/biobert-base-cased-v1.1/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\78e6e8ece5b58501028ce314273009ad7707ef4c5ba44251914fd6bca8a05eff.e4a2e693122d98b8b56b7dc1f0d89b644226aacef228afb5030ee3621b2829d3
Model config BertConfig {
  "_name_or_path": "dmis-lab/biobert-base-cased-v1.1",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading

AttributeError: type object 'EncoderDecoderConfig' has no attribute 'from_encoder_decoder_pretrained'

In [91]:
from transformers import AutoTokenizer 

In [167]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping"

In [168]:
prefix = "summarize: "

In [169]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [170]:
#!pip install datasets
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")

Found cached dataset billsum (C:/Users/AlexandruDaia/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


In [171]:
billsum = billsum.train_test_split(test_size=0.2)

In [172]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [180]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [185]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['train']['text']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples['train']['text'], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [186]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

KeyError: 'train'

In [162]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [163]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [113]:
#!pip install evaluate
import evaluate

In [114]:
rouge = evaluate.load("rouge")

In [115]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [116]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "n

In [118]:
train=df[:80]
test=df[80:]
print(train.shape)
print(test.shape)

(80, 2)
(20, 2)


In [150]:
 
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)
my_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})
my_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 80
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 20
    })
})

In [152]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
import evaluate

rouge = evaluate.load("rouge")
import evaluate

rouge = evaluate.load("rouge")

In [157]:
tokenized_billsum = my_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

TypeError: PreTrainedTokenizerBase.__call__() missing 1 required positional argument: 'text'

In [153]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=my_dict["train"],
    eval_dataset=my_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 80
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['text', 'summary']

In [123]:
DataDict()

NameError: name 'DataDict' is not defined